## Encoding Categorical Variables


**Categorical variables** are those whose values are selected from a group of categories or labels. For example, the **Home owner** variable with the values of **owner** and **non-owner** is categorical, and so is the Marital status variable with the values of never married, married, divorced, and widowed. In some categorical variables, the labels have an intrinsic order; for example, in the Student's grade variable, the values of A, B, C, and Fail are ordered, with A being the highest grade and Fail being the lowest. These are called ordinal categorical variables. Variables in which the categories do not have an intrinsic order are called nominal categorical variables, such as the City variable, with the values of London, Manchester, Bristol, and so on

The values of categorical variables are often encoded as strings. To train most machine learning models, we need to transform those strings into numbers. The act of replacing strings with numbers is called categorical encoding. In this chapter, we will discuss multiple categorical encoding methods

Creating binary variables through one-hot encoding
- Performing one-hot encoding of frequent categories
- Replacing categories with counts or the frequency of observations
- Replacing categories with ordinal numbers
Performing ordinal encoding based on the target value
- Implementing target mean encoding
- Encoding with the Weight of Evidence
- Grouping rare or infrequent categories
- Performing binary encoding

### Creating binary variables through one-hot encoding

One-hot encoding is a method used to represent categorical data, where each category is represented by a binary variable. The binary variable takes a value of 1 if the category is present, or 0 otherwise
![One hotencode](../img/one_hotencoder.png)



For the Color categorical variable with the values of red, blue, and green, we can create three variables called **red**, **blue**, and **green**. These variables will be assigned a value of 1 if the observation corresponds to the respective color, and 0 if it does not.

Encoding into k-1 binary variables is well suited for linear models. There are a few occasions in which we may prefer to encode the categorical variables with k binary variables:

- When training decision trees, since they do not evaluate the entire feature space at the same time
- When selecting features recursively
- When determining the importance of each category within a variable

In [245]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')

X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)

Let's inspect the unique categories of the A4 variable

In [246]:
X_train["A4"].unique()

array(['u', 'y', 'Missing', 'l'], dtype=object)

Let's encode A4 into k-1 binary variables using pandas and then inspect the first five rows of the resulting DataFrame:

In [247]:
dummies = pd.get_dummies(X_train['A4'])
dummies

,Missing,l,u,y
596,False,False,True,False
303,False,False,True,False
204,False,False,False,True
351,False,False,False,True
118,False,False,True,False
...,...,...,...,...
359,False,False,True,False
192,False,False,True,False
629,False,False,True,False
559,False,False,True,False


Now, let's encode all the categorical variables into k-1 binaries:

In [248]:
X_train_enc = pd.get_dummies(X_train, drop_first=True)
X_test_enc = pd.get_dummies(X_test, drop_first=True)

In [249]:
X_train_enc.head()

,A2,A3,A8,A11,A14,A15,A1_a,A1_b,A4_l,A4_u,A4_y,A5_g,A5_gg,A5_p,A6_aa,A6_c,A6_cc,A6_d,A6_e,A6_ff,A6_i,A6_j,A6_k,A6_m,A6_q,A6_r,A6_w,A6_x,A7_bb,A7_dd,A7_ff,A7_h,A7_j,A7_n,A7_o,A7_v,A7_z,A9_t,A10_t,A12_t,A13_p,A13_s
596,46.08,3.000,2.375,8,396.0,4159,True,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,True,False,False
303,15.92,2.875,0.085,0,120.0,0,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False
204,36.33,2.125,0.085,1,50.0,1187,False,True,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,True,False,False,False
351,22.17,0.585,0.000,0,100.0,0,False,True,False,False,True,False,False,True,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False
118,57.83,7.040,14.000,6,360.0,1332,False,True,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,True,False,False


**IMPORTANT**: pandas' get_dummies()will create one binary variable per category seen in a DataFrame. Hence, if there are more categories in the train set than in the test set, get_dummies() will return more columns in the transformed train set than in the transformed test set, and vice versa. To avoid this, it is better to carry out one-hot encoding with scikit-learn or feature-engine.

### Let's do one-hot encoding using scikit-learn instead.

In [250]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)

cat_vars = X_train.select_dtypes(include='O').columns.to_list()
encoder = OneHotEncoder(drop='first', sparse_output=False)

Let's restrict the encoding to the categorical variables:

In [251]:
ct = ColumnTransformer(
    [("encoder", encoder, cat_vars)],
    remainder='passthrough',
    force_int_remainder_cols=False,
).set_output(transform='pandas')

ct.fit(X_train)

ColumnTransformer(force_int_remainder_cols=False, remainder='passthrough',
                  transformers=[('encoder',
                                 OneHotEncoder(drop='first',
                                               sparse_output=False),
                                 ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10',
                                  'A12', 'A13'])])

Let's inspect the categories that will be represented with binary variables



In [252]:
ct.named_transformers_['encoder'].categories_

[array(['Missing', 'a', 'b'], dtype=object),
 array(['Missing', 'l', 'u', 'y'], dtype=object),
 array(['Missing', 'g', 'gg', 'p'], dtype=object),
 array(['Missing', 'aa', 'c', 'cc', 'd', 'e', 'ff', 'i', 'j', 'k', 'm',
        'q', 'r', 'w', 'x'], dtype=object),
 array(['Missing', 'bb', 'dd', 'ff', 'h', 'j', 'n', 'o', 'v', 'z'],
       dtype=object),
 array(['f', 't'], dtype=object),
 array(['f', 't'], dtype=object),
 array(['f', 't'], dtype=object),
 array(['g', 'p', 's'], dtype=object)]

**NOTE**: scikit-learn's OneHotEncoder() will only encode the categories learned from the train set. If there are new categories in the test set, we can instruct the encoder to ignore them, return an error, or replace them with an infrequent category, by setting the handle_unknown parameter to ignore, error, or infrequent_if_exists.

In [253]:
X_train_enc = ct.transform(X_train)
X_test_enc = ct.transform(X_test)

In [254]:
X_test_enc.head()

,encoder__A1_a,encoder__A1_b,encoder__A4_l,encoder__A4_u,encoder__A4_y,encoder__A5_g,encoder__A5_gg,encoder__A5_p,encoder__A6_aa,encoder__A6_c,encoder__A6_cc,encoder__A6_d,encoder__A6_e,encoder__A6_ff,encoder__A6_i,encoder__A6_j,encoder__A6_k,encoder__A6_m,encoder__A6_q,encoder__A6_r,encoder__A6_w,encoder__A6_x,encoder__A7_bb,encoder__A7_dd,encoder__A7_ff,encoder__A7_h,encoder__A7_j,encoder__A7_n,encoder__A7_o,encoder__A7_v,encoder__A7_z,encoder__A9_t,encoder__A10_t,encoder__A12_t,encoder__A13_p,encoder__A13_s,remainder__A2,remainder__A3,remainder__A8,remainder__A11,remainder__A14,remainder__A15
14,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,45.83,10.50,5.000,7,0.0,0
586,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,64.08,20.00,17.500,9,0.0,1000
140,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,31.25,3.75,0.625,9,181.0,0
492,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,39.25,9.50,6.500,14,240.0,4607
350,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,26.17,2.00,0.000,0,276.0,1


In [255]:
ct.get_feature_names_out()

array(['encoder__A1_a', 'encoder__A1_b', 'encoder__A4_l', 'encoder__A4_u',
       'encoder__A4_y', 'encoder__A5_g', 'encoder__A5_gg',
       'encoder__A5_p', 'encoder__A6_aa', 'encoder__A6_c',
       'encoder__A6_cc', 'encoder__A6_d', 'encoder__A6_e',
       'encoder__A6_ff', 'encoder__A6_i', 'encoder__A6_j',
       'encoder__A6_k', 'encoder__A6_m', 'encoder__A6_q', 'encoder__A6_r',
       'encoder__A6_w', 'encoder__A6_x', 'encoder__A7_bb',
       'encoder__A7_dd', 'encoder__A7_ff', 'encoder__A7_h',
       'encoder__A7_j', 'encoder__A7_n', 'encoder__A7_o', 'encoder__A7_v',
       'encoder__A7_z', 'encoder__A9_t', 'encoder__A10_t',
       'encoder__A12_t', 'encoder__A13_p', 'encoder__A13_s',
       'remainder__A2', 'remainder__A3', 'remainder__A8',
       'remainder__A11', 'remainder__A14', 'remainder__A15'], dtype=object)

### To wrap up the recipe, let's perform one-hot encoding with feature-engine

In [256]:
import pandas as pd
from sklearn.model_selection import train_test_split
from feature_engine.encoding import OneHotEncoder

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)

ohe_enc = OneHotEncoder(drop_last=True)

ohe_enc.fit(X_train)
ohe_enc.variables_

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [257]:
ohe_enc.encoder_dict_

{'A1': ['a', 'b'],
 'A4': ['u', 'y', 'Missing'],
 'A5': ['g', 'p', 'Missing'],
 'A6': ['c',
  'q',
  'w',
  'ff',
  'm',
  'i',
  'e',
  'cc',
  'x',
  'd',
  'k',
  'j',
  'Missing',
  'aa'],
 'A7': ['v', 'ff', 'h', 'dd', 'z', 'bb', 'j', 'Missing', 'n'],
 'A9': ['t'],
 'A10': ['t'],
 'A12': ['t'],
 'A13': ['g', 's']}

Let's encode the categorical variables in train and test sets:

In [258]:
X_train_enc = ohe_enc.transform(X_train)
X_test_enc = ohe_enc.transform(X_test)

In [259]:

X_train_enc.head()

,A2,A3,A8,A11,A14,A15,A1_a,A1_b,A4_u,A4_y,A4_Missing,A5_g,A5_p,A5_Missing,A6_c,A6_q,A6_w,A6_ff,A6_m,A6_i,A6_e,A6_cc,A6_x,A6_d,A6_k,A6_j,A6_Missing,A6_aa,A7_v,A7_ff,A7_h,A7_dd,A7_z,A7_bb,A7_j,A7_Missing,A7_n,A9_t,A10_t,A12_t,A13_g,A13_s
596,46.08,3.000,2.375,8,396.0,4159,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,0
303,15.92,2.875,0.085,0,120.0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
204,36.33,2.125,0.085,1,50.0,1187,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0
351,22.17,0.585,0.000,0,100.0,0,0,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0
118,57.83,7.040,14.000,6,360.0,1332,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,0


In [260]:
X_test_enc.head()

,A2,A3,A8,A11,A14,A15,A1_a,A1_b,A4_u,A4_y,A4_Missing,A5_g,A5_p,A5_Missing,A6_c,A6_q,A6_w,A6_ff,A6_m,A6_i,A6_e,A6_cc,A6_x,A6_d,A6_k,A6_j,A6_Missing,A6_aa,A7_v,A7_ff,A7_h,A7_dd,A7_z,A7_bb,A7_j,A7_Missing,A7_n,A9_t,A10_t,A12_t,A13_g,A13_s
14,45.83,10.50,5.000,7,0.0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,1,1,0
586,64.08,20.00,17.500,9,0.0,1000,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0
140,31.25,3.75,0.625,9,181.0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,1,1,0
492,39.25,9.50,6.500,14,240.0,4607,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,1,0
350,26.17,2.00,0.000,0,276.0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0


**NOTE**: feature-engine's OneHotEncoder() encodes all categorical variables by default. To encode a subset of the variables, pass the variable names in a list: OneHotEncoder(variables=["A1", "A4"]). To encode numerical variables, set the ignore_format parameter to True or cast the variables as objects.

In [261]:
ohe_enc.get_feature_names_out()

['A2',
 'A3',
 'A8',
 'A11',
 'A14',
 'A15',
 'A1_a',
 'A1_b',
 'A4_u',
 'A4_y',
 'A4_Missing',
 'A5_g',
 'A5_p',
 'A5_Missing',
 'A6_c',
 'A6_q',
 'A6_w',
 'A6_ff',
 'A6_m',
 'A6_i',
 'A6_e',
 'A6_cc',
 'A6_x',
 'A6_d',
 'A6_k',
 'A6_j',
 'A6_Missing',
 'A6_aa',
 'A7_v',
 'A7_ff',
 'A7_h',
 'A7_dd',
 'A7_z',
 'A7_bb',
 'A7_j',
 'A7_Missing',
 'A7_n',
 'A9_t',
 'A10_t',
 'A12_t',
 'A13_g',
 'A13_s']

### Performing one-hot encoding of frequent categories

One-hot encoding represents each variable's category with a binary variable. Hence, one-hot encoding of highly cardinal variables or datasets with multiple categorical features can expand the feature space dramatically. This, in turn, may increase the computational cost of using machine learning models or deteriorate their performance. To reduce the number of binary variables, we can perform one-hot encoding of the most frequent categories. One-hot encoding the top categories is equivalent to treating the remaining, less frequent categories as a single, unique category.

In [262]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)


**NOTA**: The most frequent categories need to be determined in the train set. This is to avoid data leakage.

In [263]:
X_train['A6'].unique()

array(['c', 'q', 'w', 'ff', 'm', 'i', 'e', 'cc', 'x', 'd', 'k', 'j',
       'Missing', 'aa', 'r'], dtype=object)

Let's count the number of observations per category of A6, sort them in decreasing order, and then display the five most frequent categories:

In [264]:
X_train['A6'].value_counts().sort_values(ascending=False).head()

A6
c     93
q     56
w     48
i     41
ff    38
Name: count, dtype: int64

Now, let's capture the most frequent categories of A6 in a list by using the code in Step 4 inside a list comprehension:

In [265]:
top_5 = [x for x in X_train['A6'].value_counts().sort_values(ascending=False).head(5).index]
top_5

['c', 'q', 'w', 'i', 'ff']

Let's add a binary variable per top category to a copy of the train and test sets

In [266]:
X_train_enc = X_train.copy()
X_test_enc = X_test.copy()
for label in top_5:
    X_train_enc[f"A6_{label}"] = np.where(
        X_train["A6"] == label, 1, 0)
    X_test_enc[f"A6_{label}"] = np.where(
        X_test["A6"] == label, 1, 0)

Let's display the top 10 rows of the original and encoded variable, A6, in the train set:

In [267]:
X_train_enc[["A6"] + [f"A6_{label}" for label in top_5]].head(10)

,A6,A6_c,A6_q,A6_w,A6_i,A6_ff
596,c,1,0,0,0,0
303,q,0,1,0,0,0
204,w,0,0,1,0,0
351,ff,0,0,0,0,1
118,m,0,0,0,0,0
247,q,0,1,0,0,0
652,i,0,0,0,1,0
513,e,0,0,0,0,0
230,cc,0,0,0,0,0
250,e,0,0,0,0,0


### Let's now automate one-hot encoding of frequent categories with scikit-learn

In [268]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)

encoder = OneHotEncoder(
        min_frequency=39,
        max_categories=5,
        sparse_output=False,
    ).set_output(transform='pandas')

Finally, let's fit the transformer to the two high cardinal variables and then transform the data:

In [269]:
X_train_enc = encoder.fit_transform(X_train[['A6', 'A7']])
X_test_enc = encoder.transform(X_test[['A6','A7']])

X_train_enc.head()

,A6_c,A6_i,A6_q,A6_w,A6_infrequent_sklearn,A7_bb,A7_ff,A7_h,A7_v,A7_infrequent_sklearn
596,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
303,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
204,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
351,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
118,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


### To wrap up the recipe, let's encode the most frequent categories with feature-engine.

In [270]:
import pandas as pd
from sklearn.model_selection import train_test_split
from feature_engine.encoding import OneHotEncoder

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)

ohe_enc = OneHotEncoder(
    top_categories=5,
    variables=['A6','A7']
)

ohe_enc.fit(X_train)

OneHotEncoder(top_categories=5, variables=['A6', 'A7'])

In [271]:
X_train_enc = ohe_enc.transform(X_train)
X_test_enc = ohe_enc.transform(X_test)

In [272]:
X_train_enc.head()

,A1,A2,A3,A4,A5,A8,A9,A10,A11,A12,A13,A14,A15,A6_c,A6_q,A6_w,A6_i,A6_ff,A7_v,A7_h,A7_ff,A7_bb,A7_z
596,a,46.08,3.000,u,g,2.375,t,t,8,t,g,396.0,4159,1,0,0,0,0,1,0,0,0,0
303,a,15.92,2.875,u,g,0.085,f,f,0,f,g,120.0,0,0,1,0,0,0,1,0,0,0,0
204,b,36.33,2.125,y,p,0.085,t,t,1,f,g,50.0,1187,0,0,1,0,0,1,0,0,0,0
351,b,22.17,0.585,y,p,0.000,f,f,0,f,g,100.0,0,0,0,0,0,1,0,0,1,0,0
118,b,57.83,7.040,u,g,14.000,t,t,6,t,g,360.0,1332,0,0,0,0,0,1,0,0,0,0


### Replacing categories with counts or the frequency of observations

In count with counts or frequency of observations or frequency encoding, we replace the categories with the count or the fraction of observations showing that category. That is, if 10 out of 100 observations show the blue category for the Color variable, we would replace blue with 10 when doing count encoding, or with 0.1 if performing frequency encoding. These encoding methods are useful when there is a relationship between the category frequency and the target. For example, in sales, the frequency of a product may indicate its popularity

In [273]:
import pandas as pd
from sklearn.model_selection import train_test_split
from feature_engine.encoding import CountFrequencyEncoder

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)

count_enc = CountFrequencyEncoder(
    encoding_method='count', variables=None
)

count_enc.fit(X_train)

CountFrequencyEncoder()

In [274]:
X_train_enc = count_enc.transform(X_train)
X_test_enc = count_enc.transform(X_test)

In [275]:
count_enc.variables_

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [276]:
X_train_enc.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
596,144,46.08,3.000,363,363,93,277,2.375,256,212,8,220,441,396.0,4159
303,144,15.92,2.875,363,363,56,277,0.085,227,271,0,263,441,120.0,0
204,335,36.33,2.125,115,115,48,277,0.085,256,212,1,263,441,50.0,1187
351,335,22.17,0.585,115,115,38,41,0.000,227,271,0,263,441,100.0,0
118,335,57.83,7.040,363,363,26,277,14.000,256,212,6,220,441,360.0,1332


### Replacing categories with ordinal numbers

Ordinal encoding consists of replacing the categories with digits from 1 to k (or 0 to k-1, depending on the implementation), where k is the number of distinct categories of the variable. The numbers are assigned arbitrarily. Ordinal encoding is better suited for non-linear machine learning models, which can navigate through arbitrarily assigned numbers to find patterns that relate to the target.

In [277]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'],test_size=.3, random_state=0)

To encode the A7 variable, let's make a dictionary of category-to-integer pairs

In [278]:
ordinal_mapping = {k: i for i, k in  enumerate(X_train['A7'].unique(), 0)}

ordinal_mapping

{'v': 0,
 'ff': 1,
 'h': 2,
 'dd': 3,
 'z': 4,
 'bb': 5,
 'j': 6,
 'Missing': 7,
 'n': 8,
 'o': 9}

In [279]:
X_train_enc = X_train.copy()
X_test_enc = X_test.copy()
X_train_enc['A7'] = X_train_enc['A7'].map(ordinal_mapping)
X_test_enc['A7'] = X_test_enc['A7'].map(ordinal_mapping)

X_train_enc['A7'].head()

596    0
303    0
204    0
351    1
118    0
Name: A7, dtype: int64

### Next, we'll carry out ordinal encoding using scikit-learn


In [280]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.compose import ColumnTransformer

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)

cat_vars = X_train.select_dtypes(include='O').columns.to_list()
print(cat_vars)
print(X_train[cat_vars])

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']
    A1 A4 A5  A6  A7 A9 A10 A12 A13
596  a  u  g   c   v  t   t   t   g
303  a  u  g   q   v  f   f   f   g
204  b  y  p   w   v  t   t   f   g
351  b  y  p  ff  ff  f   f   f   g
118  b  u  g   m   v  t   t   t   g
..  .. .. ..  ..  .. ..  ..  ..  ..
359  a  u  g  ff  ff  f   f   f   g
192  b  u  g   x   v  t   f   f   g
629  a  u  g   w   v  f   f   f   g
559  a  u  g   q   h  t   t   t   g
684  b  u  g   m   v  f   f   t   s

[483 rows x 9 columns]


In [281]:
# let's setup the encoder
enc = OrdinalEncoder()
ct = ColumnTransformer(
    [('encoder', enc, cat_vars)],
    remainder='passthrough',
    force_int_remainder_cols=False,
).set_output(transform='pandas')


In [282]:
ct.fit(X_train)

ColumnTransformer(force_int_remainder_cols=False, remainder='passthrough',
                  transformers=[('encoder', OrdinalEncoder(),
                                 ['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10',
                                  'A12', 'A13'])])

In [283]:
X_train_enc = ct.transform(X_train)
X_test_enc = ct.transform(X_test)

In [284]:
X_train_enc.head()

,encoder__A1,encoder__A4,encoder__A5,encoder__A6,encoder__A7,encoder__A9,encoder__A10,encoder__A12,encoder__A13,remainder__A2,remainder__A3,remainder__A8,remainder__A11,remainder__A14,remainder__A15
596,1.0,2.0,1.0,2.0,8.0,1.0,1.0,1.0,0.0,46.08,3.000,2.375,8,396.0,4159
303,1.0,2.0,1.0,11.0,8.0,0.0,0.0,0.0,0.0,15.92,2.875,0.085,0,120.0,0
204,2.0,3.0,3.0,13.0,8.0,1.0,1.0,0.0,0.0,36.33,2.125,0.085,1,50.0,1187
351,2.0,3.0,3.0,6.0,3.0,0.0,0.0,0.0,0.0,22.17,0.585,0.000,0,100.0,0
118,2.0,2.0,1.0,10.0,8.0,1.0,1.0,1.0,0.0,57.83,7.040,14.000,6,360.0,1332


In [285]:
ct.named_transformers_['encoder'].categories_

[array(['Missing', 'a', 'b'], dtype=object),
 array(['Missing', 'l', 'u', 'y'], dtype=object),
 array(['Missing', 'g', 'gg', 'p'], dtype=object),
 array(['Missing', 'aa', 'c', 'cc', 'd', 'e', 'ff', 'i', 'j', 'k', 'm',
        'q', 'r', 'w', 'x'], dtype=object),
 array(['Missing', 'bb', 'dd', 'ff', 'h', 'j', 'n', 'o', 'v', 'z'],
       dtype=object),
 array(['f', 't'], dtype=object),
 array(['f', 't'], dtype=object),
 array(['f', 't'], dtype=object),
 array(['g', 'p', 's'], dtype=object)]

### Now, let's do ordinal encoding with feature-engine

In [286]:
import pandas as pd
from sklearn.model_selection import train_test_split
from feature_engine.encoding import OrdinalEncoder

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)

cat_vars = X_train.select_dtypes(include='O').columns.to_list()
cat_vars

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [287]:
enc = OrdinalEncoder(
    encoding_method='arbitrary',
    variables=cat_vars
)



**NOTE**: feature-engine's OrdinalEncoder() automatically finds and encodes all categorical variables if the variables parameter is None. Alternatively, it will encode the variables indicated in the list. In addition, it can assign the integers according to the target mean value (see the Performing ordinal encoding based on the target value recipe)

In [288]:
enc.fit(X_train)

OrdinalEncoder(encoding_method='arbitrary',
               variables=['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12',
                          'A13'])

In [289]:
X_train_enc = enc.transform(X_train)
X_test_enc = enc.transform(X_test)

In [290]:
X_train_enc[cat_vars].head()

,A1,A4,A5,A6,A7,A9,A10,A12,A13
596,0,0,0,0,0,0,0,0,0
303,0,0,0,1,0,1,1,1,0
204,1,1,1,2,0,0,0,1,0
351,1,1,1,3,1,1,1,1,0
118,1,0,0,4,0,0,0,0,0


### Performing ordinal encoding based on the target value

In the previous recipe, we replaced categories with integers, which were assigned arbitrarily. We can also assign integers to the categories given the target values. To do this, first, we calculate the mean value of the target per category. Next, we order the categories from the one with the lowest to the one with the highest target mean value. Finally, we assign digits to the ordered categories, starting with 0 to the first category up to k-1 to the last category, where k is the number of distinct categories.

In [291]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)

Let's determine the mean target value per category in A7, then sort the categories from that with the lowest to that with the highest target value

In [292]:
y_train.groupby(X_train['A7']).mean().sort_values(ascending=False)

A7
Missing    1.000000
z          0.714286
n          0.666667
h          0.603960
bb         0.512821
v          0.418773
dd         0.400000
j          0.200000
ff         0.146341
o          0.000000
Name: target, dtype: float64

Now, let's repeat the computation in Step 3, but this time, let's retain the ordered category names:

In [293]:
ordered_labels = y_train.groupby(X_train['A7']).mean().sort_values().index
ordered_labels

Index(['o', 'ff', 'j', 'dd', 'v', 'bb', 'h', 'n', 'z', 'Missing'], dtype='object', name='A7')

Let's create a dictionary of category-to-integer pairs, using the ordered list we created in Step 4

In [294]:
ordinal_mapping = {k:i for i, k in enumerate(ordered_labels, 0)}
ordinal_mapping

{'o': 0,
 'ff': 1,
 'j': 2,
 'dd': 3,
 'v': 4,
 'bb': 5,
 'h': 6,
 'n': 7,
 'z': 8,
 'Missing': 9}

In [295]:
X_train_enc = X_train.copy()
X_test_enc = X_test.copy()
X_test_enc["A7"] = X_train_enc['A7'].map(ordinal_mapping)
X_test_enc['A7'] = X_test_enc['A7'].map(ordinal_mapping)

### Now, let's perform ordered ordinal encoding using feature-engine.

In [301]:
import pandas as pd
from sklearn.model_selection import train_test_split
from feature_engine.encoding import OrdinalEncoder

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)

ordinal_enc = OrdinalEncoder(
    encoding_method='ordered',
    variables=None
)

ordinal_enc.fit(X_train, y_train)

X_train_enc = ordinal_enc.transform(X_train)
X_test_enc = ordinal_enc.transform(X_test)

ordinal_enc.encoder_dict_

{'A1': {'b': 0, 'a': 1, 'Missing': 2},
 'A4': {'y': 0, 'u': 1, 'Missing': 2, 'l': 3},
 'A5': {'p': 0, 'g': 1, 'Missing': 2, 'gg': 3},
 'A6': {'ff': 0,
  'd': 1,
  'j': 2,
  'i': 3,
  'k': 4,
  'm': 5,
  'aa': 6,
  'c': 7,
  'w': 8,
  'e': 9,
  'q': 10,
  'cc': 11,
  'x': 12,
  'Missing': 13,
  'r': 14},
 'A7': {'o': 0,
  'ff': 1,
  'j': 2,
  'dd': 3,
  'v': 4,
  'bb': 5,
  'h': 6,
  'n': 7,
  'z': 8,
  'Missing': 9},
 'A9': {'f': 0, 't': 1},
 'A10': {'f': 0, 't': 1},
 'A12': {'t': 0, 'f': 1},
 'A13': {'s': 0, 'g': 1, 'p': 2}}

In [302]:
X_train_enc.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
596,1,46.08,3.000,1,1,7,4,2.375,1,1,8,0,1,396.0,4159
303,1,15.92,2.875,1,1,10,4,0.085,0,0,0,1,1,120.0,0
204,0,36.33,2.125,0,0,8,4,0.085,1,1,1,1,1,50.0,1187
351,0,22.17,0.585,0,0,0,1,0.000,0,0,0,1,1,100.0,0
118,0,57.83,7.040,1,1,5,4,14.000,1,1,6,0,1,360.0,1332


#### Implementing target mean encoding

Mean encoding or target encoding maps each category to the probability estimate of the target attribute. If the target is binary, the numerical mapping is the posterior probability of the target conditioned to the value of the category. If the target is continuous, the numerical representation is given by the expected value of the target given the value of the category.

In its simplest form, the numerical representation for each category is given by the mean value of the target variable for a particular category group. For example, if we have a City variable, with the categories of London, Manchester, and Bristol, and we want to predict the default rate (the target takes values of 0 and 1); if the default rate for London is 30%, we replace London with 0.3; if the default rate for Manchester is 20%, we replace Manchester with 0.2; and so on. If the target is continuous - say we want to predict income - then we would replace London, Manchester, and Bristol with the mean income earned in each city.

### Grouping rare or infrequent categories

Rare categories are those present only in a small fraction of the observations. There is no rule of thumb to determine how small a small fraction is, but typically, any value below 5% can be considered rare.

Infrequent labels often appear only on the train set or only on the test set, thus making the algorithms prone to overfitting or being unable to score an observation. In addition, when encoding categories to numbers, we only create mappings for those categories observed in the train set, so we won't know how to encode new labels. To avoid these complications, we can group infrequent categories into a single category called Rare or Other

In [304]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)

Let's capture the fraction of observations per category in A7 in a variable

In [311]:
freqs = X_train['A7'].value_counts(normalize=True)
freqs

A7
v          0.573499
h          0.209110
ff         0.084886
bb         0.080745
z          0.014493
dd         0.010352
j          0.010352
Missing    0.008282
n          0.006211
o          0.002070
Name: proportion, dtype: float64

Let's create a list containing the names of the categories present in more than 5% of the observations:

In [313]:
frequent_cat = [
    x for x in freqs.loc[freqs > 0.05].index.values]
frequent_cat

['v', 'h', 'ff', 'bb']

Let's replace rare labels - that is, those present in <= 5% of the observations - with the Rare string in a copy of the datasets:

In [315]:
X_train_enc = X_train.copy()
X_test_enc = X_test.copy()
X_train_enc['A7'] = np.where(X_train_enc['A7'].isin(frequent_cat), X_train['A7'], 'Rare')
X_test_enc['A7'] = np.where(X_test_enc['A7'].isin(frequent_cat), X_test['A7'], 'Rare')

X_test_enc['A7']

14        v
586       h
140       h
492       v
350    Rare
       ... 
188       v
380      bb
369    Rare
362      ff
90        h
Name: A7, Length: 207, dtype: object

Let's determine the percentage of observations in the encoded variable:

In [317]:
X_train_enc['A7'].value_counts(normalize=True)

A7
v       0.573499
h       0.209110
ff      0.084886
bb      0.080745
Rare    0.051760
Name: proportion, dtype: float64

### Now, let's group rare labels using feature-engine.

In [ ]:

import warnings
import pandas as pd
from sklearn.model_selection import train_test_split
from feature_engine.encoding import RareLabelEncoder

warnings.filterwarnings('ignore')

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)

rare_encoder = RareLabelEncoder(tol=0.05, n_categories=4)



rare_encoder.fit(X_train)


RareLabelEncoder(n_categories=4)

In [324]:
rare_encoder.variables_

['A1', 'A4', 'A5', 'A6', 'A7', 'A9', 'A10', 'A12', 'A13']

In [325]:
rare_encoder.encoder_dict_

{'A1': ['a', 'b', 'Missing'],
 'A4': ['u', 'y', 'Missing', 'l'],
 'A5': ['g', 'p', 'Missing', 'gg'],
 'A6': ['c', 'q', 'w', 'i', 'ff', 'k', 'aa', 'cc', 'm'],
 'A7': ['v', 'h', 'ff', 'bb'],
 'A9': ['t', 'f'],
 'A10': ['t', 'f'],
 'A12': ['t', 'f'],
 'A13': ['g', 's', 'p']}

In [328]:
X_train_enc = rare_encoder.transform(X_train)
X_test_enc = rare_encoder.transform(X_test)

X_train_enc.head(5)

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15
596,a,46.08,3.000,u,g,c,v,2.375,t,t,8,t,g,396.0,4159
303,a,15.92,2.875,u,g,q,v,0.085,f,f,0,f,g,120.0,0
204,b,36.33,2.125,y,p,w,v,0.085,t,t,1,f,g,50.0,1187
351,b,22.17,0.585,y,p,ff,ff,0.000,f,f,0,f,g,100.0,0
118,b,57.83,7.040,u,g,m,v,14.000,t,t,6,t,g,360.0,1332


### Performing binary encoding

Binary encoding uses binary code - that is, a sequence of zeroes and ones - to represent the different categories of the variable. How does it work? First, the categories are arbitrarily replaced with ordinal numbers, as shown in the intermediate step of the following table. Then, those numbers are converted into binary code. For example, integer 1 can be represented with the sequence of 1-0, integer 2 with 0-1, integer 3 with 1-1, and integer 0 with 0-0. The digits in the two positions of the binary string become the columns, which are the encoded representations of the original variable:



In [330]:
import pandas as pd
from sklearn.model_selection import train_test_split
from category_encoders.binary import BinaryEncoder # type: ignore

data = pd.read_csv('../source/ch02-categorical-encoding/credit_approval_uci.csv')
X_train, X_test, y_train, y_test = train_test_split(data.drop('target', axis=1), data['target'], test_size=.3, random_state=0)

# Let's inspect the unique  categories in A7
X_train['A7'].unique()



array(['v', 'ff', 'h', 'dd', 'z', 'bb', 'j', 'Missing', 'n', 'o'],
      dtype=object)

In [332]:
# Let's create a binary encoder to encode A7
encoder = BinaryEncoder(cols=['A7'], drop_invariant=True)
encoder.fit(X_train)

BinaryEncoder(cols=['A7'], drop_invariant=True,
              mapping=[{'col': 'A7',
                        'mapping':      A7_0  A7_1  A7_2  A7_3
 1      0     0     0     1
 2      0     0     1     0
 3      0     0     1     1
 4      0     1     0     0
 5      0     1     0     1
 6      0     1     1     0
 7      0     1     1     1
 8      1     0     0     0
 9      1     0     0     1
 10     1     0     1     0
-1      0     0     0     0
-2      0     0     0     0}])

In [335]:
X_train_enc = encoder.transform(X_train)
X_test_enc = encoder.transform(X_test)

In [336]:
X_train_enc.head()

,A1,A2,A3,A4,A5,A6,A7_0,A7_1,A7_2,A7_3,A8,A9,A10,A11,A12,A13,A14,A15
596,a,46.08,3.000,u,g,c,0,0,0,1,2.375,t,t,8,t,g,396.0,4159
303,a,15.92,2.875,u,g,q,0,0,0,1,0.085,f,f,0,f,g,120.0,0
204,b,36.33,2.125,y,p,w,0,0,0,1,0.085,t,t,1,f,g,50.0,1187
351,b,22.17,0.585,y,p,ff,0,0,1,0,0.000,f,f,0,f,g,100.0,0
118,b,57.83,7.040,u,g,m,0,0,0,1,14.000,t,t,6,t,g,360.0,1332
